# **Eratos access and Library Import**

In [ ]:
%%capture
#Install Eratos SDK
!pip install https://releases.eratos.com/sdk/python/eratos-python-latest.zip
!pip install geopandas
#!pip install KeplerGl
print('The Eratos SDK is installed')

In [ ]:
#Import libraries
from eratos.creds import AccessTokenCreds
from eratos.adapter import Adapter
from getpass import getpass
from google.colab import output
output.enable_custom_widget_manager()

import pandas as pd
from tabulate import tabulate
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Access Eratos System
eratos_id = getpass('Enter the eratos key id:')
eratos_secret = getpass('Enter the eratos secret:')

ecreds = AccessTokenCreds(
  eratos_id,
  eratos_secret
)

eadapter = Adapter(ecreds)

print('You have successfully entered key and secret, you can now continue on below')

Enter the eratos key id:··········
Enter the eratos secret:··········
You have successfully entered key and secret, you can now continue on below


In [ ]:
#Import Dataset
# Request acccess to the data resource in Eratos
e_data = eadapter.Resource(ern='ern:e-pn.io:resource:eratos.blocks.silo.mintemperature')

# Access the gridded data via the gridded data adapter:
gridded_min_temperature_data = e_data.data().gapi()

#Threshold Less Than


In [ ]:
#Set Model Variables 
dataset = gridded_min_temperature_data #Dataset to analyse
variable = 'min_temp' #Desired variable
location = 'POINT(141.181121 -34.147659)' #Point of interest
threshold = -1 #Set variable threshold value
target_start_date =  "04-01" #MM-DD
target_end_date = "09-30" #MM-DD
start_date = "1991-01-01" #Start date for anaysis 
end_date = "2021-12-31" #End date for anaysis 


In [ ]:
import pandas as pd
import numpy as np
from datetime import  timezone
from datetime import date

def AnalyseTempThreshold_Q(dataset,variable,location,threshold,target_start_date,target_end_date,start_date,end_date):
  

  #set up local variables
  events = 0
  
  #Extract data at required location and time period
  extracted_data = (dataset.get_timeseries_at_points(variable, [location], start_date, end_date))

  #Generate years of interest to loop through
  date_generated_list = pd.date_range(start_date, end_date, freq="Y")
  date_range_years = date_generated_list.strftime("%Y").to_list()
  #print(date_range_years)
  count = 0 

  for year in date_range_years:
    #Work out how many days in a year
    d0 = date(int(year), 1, 1)
    d1 = date(int(year) + 1, 1, 1)
    year_delta = d1 - d0

    #Extract year out of full data vector
    year_arr = extracted_data[0][count:count+year_delta.days]
    # add length of year (365,366) to count so next loop pulls out the following year
    count += year_delta.days

    #Create extraction date for the year
    target_d0 = date(int(year), int(target_start_date[0:2]), int(target_start_date[3:5]))
    target_d1 = date(int(year), int(target_end_date[0:2]), int(target_end_date[3:5]))

    # Minus start of year from start/end of target period to allow target_start_delta.days below    
    target_start_delta = target_d0 - d0
    target_end_delta = target_d1 - d0


    #print(target_start_delta.days,target_end_delta.days)

    # Extract the vector that represents the target period
    target_period_arr = year_arr[target_start_delta.days:target_end_delta.days]
    bool_target_period = target_period_arr <= threshold

    #print(target_period_arr)
    #print(bool_target_period.sum())
    if bool_target_period.sum() > 0:
        events += 1

  return events/len(date_range_years)

In [ ]:
start_dates = ["1981-01-01","1991-01-01","2001-01-01"]
years = [40,30,20]
for idx, start_date in enumerate(start_dates):

  print(AnalyseTempThreshold_Q(dataset,variable,location,threshold,target_start_date,target_end_date,start_date,end_date),"years:",years[idx])





0.43902439024390244 years: 40
0.5806451612903226 years: 30
0.6190476190476191 years: 20
